In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import seaborn as sns

sns.set_theme(style="ticks")

In [ ]:
load = pd.read_parquet("../data/energy.parquet").asfreq("h")[["load"]].dropna()

In [ ]:
load

In [ ]:
sns.lineplot(load)

In [ ]:
sns.lineplot(load.query("date.dt.year == 2023"))

## Seasonalities

In [ ]:
# groupby month and show grouped boxplot
load["month"] = load.index.month
load["day"] = load.index.day
load["hour"] = load.index.hour
load["weekday"] = load.index.weekday
sns.boxplot(x="month", y="load", data=load)

In [ ]:
sns.boxplot(x="weekday", y="load", data=load)

In [ ]:
sns.boxplot(x="hour", y="load", data=load)

In [ ]:
load["load"].rolling(52 * 7 * 24).mean().plot()

In [ ]:
from sktime.utils.plotting import plot_correlations

plot_correlations(load["load"], lags=24 * 7 * 2, acf_fft=False)